In [379]:
import pandas as pd
import os

In [380]:
os.getcwd()

'/Users/jbrable/Documents/ENSAE_2AD/S1/python/UFO'

In [381]:
df = pd.read_csv("data/ufo_sightings_scrubbed.csv")

/var/folders/j3/7p5c8ygd7yg6bgkb6r90ddhm0000gn/T/ipykernel_96890/2315484744.py:1: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/ufo_sightings_scrubbed.csv")


In [382]:
df.head()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,1949-10-10 20:30:00,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,2004-04-27,29.8830556,-97.941111
1,1949-10-10 21:00:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,2005-12-16,29.38421,-98.581082
2,1955-10-10 17:00:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,2008-01-21,53.2,-2.916667
3,1956-10-10 21:00:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,2004-01-17,28.9783333,-96.645833
4,1960-10-10 20:00:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,2004-01-22,21.4180556,-157.803611


In [383]:
df.shape

(80332, 11)

In [384]:
df["shape"].value_counts()

light        16565
triangle      7865
circle        7608
fireball      6208
other         5649
unknown       5584
sphere        5387
disk          5213
oval          3733
formation     2457
cigar         2057
changing      1962
flash         1328
rectangle     1297
cylinder      1283
diamond       1178
chevron        952
egg            759
teardrop       750
cone           316
cross          233
delta            7
round            2
crescent         2
pyramid          1
flare            1
hexagon          1
dome             1
changed          1
Name: shape, dtype: int64

In [385]:
df["datetime"].sort_values(ascending = False)

46302    2014-05-08 18:45:00
46301    2014-05-08 00:00:00
46155    2014-05-07 23:30:00
46154    2014-05-07 21:20:00
46153    2014-05-07 21:10:00
                ...         
46694    1920-06-11 21:00:00
40275    1916-04-05 13:00:00
46858    1910-06-01 15:00:00
10580    1910-01-02 00:00:00
7863     1906-11-11 00:00:00
Name: datetime, Length: 80332, dtype: object

In [386]:
df["comments"].head()

0    This event took place in early fall around 194...
1    1949 Lackland AFB&#44 TX.  Lights racing acros...
2    Green/Orange circular disc over Chester&#44 En...
3    My older brother and twin sister were leaving ...
4    AS a Marine 1st Lt. flying an FJ4B fighter/att...
Name: comments, dtype: object

# Cleaning data

In [387]:
df.columns

Index(['datetime', 'city', 'state', 'country', 'shape', 'duration (seconds)',
       'duration (hours/min)', 'comments', 'date posted', 'latitude',
       'longitude '],
      dtype='object')

We notice a first problem, that is, the 'longitude' column label is mispelled : it contains a blank space at the end of the string. Let's remove it in order to avoid further issues:

In [388]:
df = df.rename(columns = {'longitude ': 'longitude'})
#df.columns

In [389]:
df.dtypes

datetime                 object
city                     object
state                    object
country                  object
shape                    object
duration (seconds)       object
duration (hours/min)     object
comments                 object
date posted              object
latitude                 object
longitude               float64
dtype: object

We also notice that 'longitude' column is a type float, which is pretty logical, whereas 'latitude' column if typed as an object : why ? Let's check it out:

In [390]:
df[df['latitude'].astype(str).str.contains('[a-zA-Z]')]

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
43782,1974-05-22 05:30:00,mescalero indian reservation,nm,NaN,rectangle,180,two hours,Huge rectangular object emmitting intense whit...,2012-04-18,33q.200088,-105.624152


We see that this problem is probably due to a typo: the corresponding latitude for 'mescalero indian reservation' contains a letter ('q'). Since this issue only concerns a single row, we can check the real latitude of this place on the Internet (if there were too many rows, we would have chosen another soluton). It turns out that the real latitude is 33.2 so the error was indeed the result of a typo, so we can remove the misplaced letter:

In [391]:
df['latitude'] = pd.to_numeric(df['latitude'].astype(str).str.extract('(\d+.\d+|\d+)', expand = False))
#df.dtypes #now both of the 'latitude' and 'longitude' columns have the right type!

Since we have clean our data, we should export it into a new .csv file, so as to facilitate oru collaborative work:

In [392]:
df.to_csv("data/ufo_sightings_clean.csv", index = False)